# Imports

In [1]:
import os, sys
sys.path.append('..') # Parent directory in path
from time import time, sleep
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import scipy.stats as stats
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
import statsmodels.api as sm

pd.set_option("display.precision", 4)
#import pandas_market_calendars as mcal # NYSE Calendar

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
plt.rc("font", **{"size": 14})
plt.rc("figure", **{"figsize": (16,10)})
# import matplotlib.pylab as pl
from matplotlib import cm
from functions.data import add_attributes

# Data
### Pick a specific timestamp

In [20]:
all_data = pd.read_hdf(os.path.join("..", "processed_data", "spx_iv_db.h5"))
ts_date = "2017-12-15"
df = all_data[all_data["TS"]==ts_date].dropna().reset_index(drop=True)
df

,TS,EXP,STRIKE,TYPE,BID_CLOSE,MID_CLOSE,ASK_CLOSE,VOLUME,OPEN_INT,UNDERLYING_PRICE,DIFF,RANK,BUS_DAYS,CAL_DAYS,RATE_OLS,RATE_OPEN_INT,RATE_VOLUME,RATE_MONEYNESS,F_T_OLS,F_T_OPEN_INT,F_T_VOLUME,F_T_MONEYNESS,RATE,F_T,IV_BID,IV_MID,IV_ASK,IMP_VOLATILITY,DELTA,GAMMA,THETA,VEGA
0,2017-12-15,2018-01-19,2675,P,19.2,19.65,20.1,9086,17945,2675.71,0.71,0,22,35,0.0246,0.0229,0.0246,0.0261,2682.2326,2682.1786,2682.2133,2682.2333,0.0229,2682.1786,0.0717,0.0732,0.0746,0.0661,-0.4664,0.0073,-0.2683,3.2622
1,2017-12-15,2018-01-19,2675,C,26.4,26.90,27.4,13300,34503,2675.71,0.71,0,22,35,0.0246,0.0229,0.0246,0.0261,2682.2326,2682.1786,2682.2133,2682.2333,0.0229,2682.1786,0.0718,0.0734,0.0750,0.0661,0.5336,0.0073,-0.3645,3.2622
2,2017-12-15,2018-01-19,2680,P,21.0,21.50,22.0,348,1586,2675.71,4.29,1,22,35,0.0246,0.0229,0.0246,0.0261,2682.2326,2682.1786,2682.2133,2682.2333,0.0229,2682.1786,0.0700,0.0716,0.0731,0.0642,-0.5035,0.0076,-0.2567,3.2737
3,2017-12-15,2018-01-19,2680,C,23.3,23.80,24.3,3919,14212,2675.71,4.29,1,22,35,0.0246,0.0229,0.0246,0.0261,2682.2326,2682.1786,2682.2133,2682.2333,0.0229,2682.1786,0.0704,0.0720,0.0735,0.0661,0.4968,0.0074,-0.3622,3.2737
4,2017-12-15,2018-01-19,2670,C,29.7,30.20,30.7,6357,2552,2675.71,5.71,2,22,35,0.0246,0.0229,0.0246,0.0261,2682.2326,2682.1786,2682.2133,2682.2333,0.0229,2682.1786,0.0734,0.0751,0.0767,0.0661,0.5700,0.0072,-0.3642,3.2232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
785,2017-12-15,2019-01-18,3100,C,8.1,9.75,11.4,0,194,2675.71,424.29,28,273,399,0.0194,0.0197,0.0197,0.0195,2690.6600,2690.7435,2690.7845,2690.6819,0.0197,2690.7435,0.0914,0.0955,0.0992,0.0986,0.0808,0.0005,-0.0505,4.1715
786,2017-12-15,2019-01-18,3100,P,405.1,410.15,415.2,0,24,2675.71,424.29,28,273,399,0.0194,0.0197,0.0197,0.0195,2690.6600,2690.7435,2690.7845,2690.6819,0.0197,2690.7435,0.0807,0.0950,0.1058,0.0760,-0.9514,0.0003,-0.0132,2.0503
787,2017-12-15,2019-01-18,2250,P,47.1,49.75,52.4,0,501,2675.71,425.71,29,273,399,0.0194,0.0197,0.0197,0.0195,2690.6600,2690.7435,2690.7845,2690.6819,0.0197,2690.7435,0.1897,0.1937,0.1976,0.1880,-0.1626,0.0005,-0.1610,6.8351
788,2017-12-15,2019-01-18,2250,C,476.1,480.95,485.8,0,0,2675.71,425.71,29,273,399,0.0194,0.0197,0.0197,0.0195,2690.6600,2690.7435,2690.7845,2690.6819,0.0197,2690.7435,0.1860,0.1933,0.2005,0.2096,0.8017,0.0004,-0.1687,7.3579


In [21]:
# Add ATM_VOLS

ATM_vols = df[df["RANK"]==0]
df = pd.merge(df, ATM_vols[["TS", "TYPE", "IV_BID", "IV_ASK"]], left_on=["TS", "TYPE"], right_on=["TS", "TYPE"], 
              suffixes=(None, "_ATM"), how="left")

# Add useful columns
df["MONEYNESS"] = df["UNDERLYING_PRICE"] - df["STRIKE"]

df["MONEYNESS_F"] = df["STRIKE"]/df["F_T"]
df["LOG_MONEYNESS_F"] = np.log(df["MONEYNESS_F"])
df["LOG_MONEYNESS_F_STANDARD_TIME"] = df["LOG_MONEYNESS_F"] / np.sqrt(df["CAL_DAYS"])

# df["LOG_MONEYNESS_F_STANDARD_STD"] = df["LOG_MONEYNESS_F"] / (np.sqrt(df["CAL_DAYS"])*(df["IV_BID_ATM"]+df["IV_ASK_ATM"])/2)

df["IV_BID_DIFF"] = df["IV_BID"] - df["IV_BID_ATM"]
df["IV_ASK_DIFF"] = df["IV_ASK"] - df["IV_ASK_ATM"]

df["IV_BID_RATIO"] = df["IV_BID"]/df["IV_BID_ATM"]
df["IV_ASK_RATIO"] = df["IV_ASK"]/df["IV_ASK_ATM"]

# Parameters
fit_data = pd.read_excel(os.path.join("..", "processed_data", "iv_parameter_TS.xlsx"), sheet_name=ts_date, index_col=0)

# Features

## Calculate features

In [8]:
def feature_avg(df, columns, weight=False):
    if not weight:
        features = df[columns].mean()
        return features
    
    else:
        wt = df[weight]
        features = df[columns].T @ wt / wt.sum()
        return features

In [22]:
feat = df.groupby("EXP").apply(feature_avg, columns=["DELTA", "GAMMA", "THETA", "VEGA", "CAL_DAYS"])
feat

,DELTA,GAMMA,THETA,VEGA,CAL_DAYS
EXP,,,,,
2018-01-19,0.1264,0.0021,-0.0969,0.9108,35.0
2018-02-16,0.2174,0.0017,-0.0859,1.4389,63.0
2018-03-16,0.1661,0.0021,-0.1337,3.0100,91.0
2018-06-15,0.1444,0.0012,-0.0898,3.8978,182.0
2018-09-21,0.0891,0.0010,-0.1196,6.2509,280.0
2018-12-21,0.0589,0.0009,-0.1145,7.7974,371.0
2019-01-18,0.0395,0.0008,-0.1567,8.9841,399.0


In [19]:
feat = df.groupby("EXP").apply(feature_avg, columns=["DELTA", "GAMMA", "THETA", "VEGA", "CAL_DAYS"])
feat

,DELTA,GAMMA,THETA,VEGA,CAL_DAYS
EXP,,,,,
2018-01-19,0.1007,0.0022,-0.1047,0.8974,32.0
2018-02-16,0.2081,0.0017,-0.0916,1.4429,60.0
2018-03-16,0.2001,0.0021,-0.1343,2.8927,88.0
2018-06-15,0.1600,0.0012,-0.0957,3.9310,179.0
2018-09-21,0.1090,0.0010,-0.1182,6.2318,277.0
2018-12-21,0.0702,0.0009,-0.1171,7.8147,368.0
2019-01-18,0.0368,0.0008,-0.1573,8.9201,396.0
